In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
import torch.nn as nn
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from typing import List
import transformers

In [ ]:
class testModel:
    def __init__(self, model, tokenizer, description):
        self.model = model
        self.description = description
        self.tokenizer = tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "ai-forever/rugpt3large_based_on_gpt2",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")

#testModel1 = testModel(baseModel, baseTokenizer, "Модель без дообучения")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=300, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 44236800 || all params: 804536832 || trainable%: 5.4984182501665755


In [ ]:
string_ = 'Луна́ — единственный естественный спутник Земли. Самый близкий к Солнцу спутник планеты, так как у ближайших к Солнцу планет (Меркурия и Венеры) их нет. Второй по яркости[комм. 1] объект на земном небосводе после Солнца и пятый по величине естественный спутник планеты Солнечной системы. Среднее расстояние между центрами Земли и Луны — 384 467 км (0,00257 а.е., ~30 диаметров Земли). Видимая звёздная величина полной Луны на земном небе — −12,71m[5]. Освещённость, создаваемая полной Луной возле поверхности Земли при ясной погоде, составляет 0,25—1 лк.Луна появилась около 4,5 млрд лет назад, немного позже Земли. Наиболее популярна гипотеза о том, что Луна сформировалась из осколков, оставшихся после «Гигантского столкновения» Земли и Тейи — планеты, схожей по размерам с Марсом.На сегодняшний день Луна является единственным внеземным астрономическим объектом, на котором побывал человек.'

In [ ]:
data = ['Луна́ — единственный естественный спутник Земли. Самый близкий к Солнцу спутник планеты, так как у ближайших к Солнцу планет (Меркурия и Венеры) их нет. Второй по яркости[комм. 1] объект на земном небосводе после Солнца и пятый по величине естественный спутник планеты Солнечной системы. Среднее расстояние между центрами Земли и Луны — 384 467 км (0,00257 а.е., ~30 диаметров Земли).','Видимая звёздная величина полной Луны на земном небе — −12,71m[5]. Освещённость, создаваемая полной Луной возле поверхности Земли при ясной погоде, составляет 0,25—1 лк.','Луна появилась около 4,5 млрд лет назад, немного позже Земли. Наиболее популярна гипотеза о том, что Луна сформировалась из осколков, оставшихся после «Гигантского столкновения» Земли и Тейи — планеты, схожей по размерам с Марсом.','На сегодняшний день Луна является единственным внеземным астрономическим объектом, на котором побывал человек.']

In [ ]:
string = "Научные данные указывают на то, что Земля образовалась из солнечной туманности около 4,54 миллиарда лет назадПерейти к разделу «#История Земли»[19] и вскоре после этого обрела свой единственный естественный спутник — Луну. Жизнь, предположительно, появилась на Земле примерно 4,25 млрд лет назад[20], то есть вскоре после её возникновенияПерейти к разделу «#Возникновение жизни». С тех пор биосфера Земли значительно изменила атмосферу и прочие абиотические факторы, обусловив количественный рост аэробных организмов, а также формирование озонового слоя, который вместе с магнитным полем Земли ослабляет вредную для жизни солнечную радиацию[21], тем самым сохраняя условия существования жизни на Земле. Радиация, обусловленная самой земной корой, со времён её образования значительно снизилась благодаря постепенному распаду радионуклидов, содержавшихся в ней. Кора Земли разделена на несколько сегментов, или тектонических плит, которые движутся по поверхности со скоростями порядка нескольких сантиметров в год. Изучением состава, строения и закономерностей развития Земли занимается наука геологияПерейти к разделу «#Внутреннее строение»."

In [ ]:
st_ = string.split('.')

In [ ]:
for item in st_:
    data.append(item)

In [ ]:
data

['Луна́ — единственный естественный спутник Земли. Самый близкий к Солнцу спутник планеты, так как у ближайших к Солнцу планет (Меркурия и Венеры) их нет. Второй по яркости[комм. 1] объект на земном небосводе после Солнца и пятый по величине естественный спутник планеты Солнечной системы. Среднее расстояние между центрами Земли и Луны — 384 467 км (0,00257 а.е., ~30 диаметров Земли).',
 'Видимая звёздная величина полной Луны на земном небе — −12,71m[5]. Освещённость, создаваемая полной Луной возле поверхности Земли при ясной погоде, составляет 0,25—1 лк.',
 'Луна появилась около 4,5 млрд лет назад, немного позже Земли. Наиболее популярна гипотеза о том, что Луна сформировалась из осколков, оставшихся после «Гигантского столкновения» Земли и Тейи — планеты, схожей по размерам с Марсом.',
 'На сегодняшний день Луна является единственным внеземным астрономическим объектом, на котором побывал человек.',
 'Научные данные указывают на то, что Земля образовалась из солнечной туманности около 

In [ ]:
data = data[:-1]

In [ ]:
data

['Луна́ — единственный естественный спутник Земли. Самый близкий к Солнцу спутник планеты, так как у ближайших к Солнцу планет (Меркурия и Венеры) их нет. Второй по яркости[комм. 1] объект на земном небосводе после Солнца и пятый по величине естественный спутник планеты Солнечной системы. Среднее расстояние между центрами Земли и Луны — 384 467 км (0,00257 а.е., ~30 диаметров Земли).',
 'Видимая звёздная величина полной Луны на земном небе — −12,71m[5]. Освещённость, создаваемая полной Луной возле поверхности Земли при ясной погоде, составляет 0,25—1 лк.',
 'Луна появилась около 4,5 млрд лет назад, немного позже Земли. Наиболее популярна гипотеза о том, что Луна сформировалась из осколков, оставшихся после «Гигантского столкновения» Земли и Тейи — планеты, схожей по размерам с Марсом.',
 'На сегодняшний день Луна является единственным внеземным астрономическим объектом, на котором побывал человек.',
 'Научные данные указывают на то, что Земля образовалась из солнечной туманности около 

In [ ]:
MAX_TOKENS = 2048
#dataset = data.map(lambda samples: tokenizer(samples['text'], max_length=MAX_TOKENS, truncation=True), batched=True)

In [ ]:
dataset = list(map(lambda samples: tokenizer(samples, max_length=MAX_TOKENS, truncation=True),data))

In [ ]:
dataset

[{'input_ids': [1098, 13145, 2442, 510, 8970, 35509, 27203, 9576, 18, 17102, 29381, 294, 9931, 1159, 27203, 9309, 16, 453, 428, 329, 24083, 294, 9931, 1159, 10863, 473, 780, 15676, 1183, 289, 7384, 1952, 13, 697, 1003, 18, 7409, 334, 3101, 7079, 63, 635, 277, 18, 430, 65, 12035, 309, 1365, 547, 1942, 397, 833, 263, 1011, 22497, 289, 23912, 334, 35942, 35509, 27203, 9309, 41374, 2926, 18, 6765, 968, 12295, 1251, 1673, 2661, 9576, 289, 26664, 510, 9690, 24, 12151, 27, 4017, 473, 20, 16, 646, 4691, 27, 367, 18, 263, 2211, 39759, 3161, 12123, 483, 9576, 814], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [28526, 13713, 34939, 549, 35037, 6409, 26664, 309, 1365, 547, 9516, 510, 24805, 2783, 16, 12630, 81, 63, 25, 3

In [ ]:
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        #fp16=True,
        logging_steps=10,
        output_dir='outputs_one_promt',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [ ]:
def generate(models : List[testModel], promt : str) -> None:
    for model in models:
        ans = promt
        MAX_TOKENS = 2048
        print(model.description)
        batch = model.tokenizer(ans, return_tensors='pt').to("cuda")
        with torch.cuda.amp.autocast():
            output_tokens = model.model.generate(**batch, max_new_tokens=75, repetition_penalty = 1.2)
            ans = " " +  model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
            print(ans, "\n")
        print("\n")

In [ ]:
testModel1 = testModel(model, tokenizer, "Дообученная модель")

In [ ]:
testModels = [testModel1]

In [ ]:
generate(testModels, "Расстояние от земли до луны")
print("\n")

Дообученная модель


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


 Расстояние от земли до луны составляет около 4,5 км.
В этом месте находится самый большой в мире кратер на Луне — Каньон Маунтин-Маунтен (Canyon Mountain Mountains). Он имеет диаметр более 3 километров и является самым большим из всех известных на Земле. Его глубина достигает почти 2 километра. В центре этого гигантского кратера расположен огромный естественный бассейн 





